# Thomas Algorithm for Tridiagonal Systems

We consider the linear system

\begin{equation}
A x = d,
\end{equation}

where $\mathbf{A}$ is a **tridiagonal matrix** with subdiagonal $a_i$, diagonal
$b_i$, and superdiagonal $c_i$:

\begin{equation}
\mathbf{A}
=
\begin{bmatrix}
b_0 & c_0 &        &        & 0 \\
a_1 & b_1 & c_1    &        &   \\
    & a_2 & b_2    & c_2    &   \\
    &     & \ddots & \ddots & c_{n-2} \\
0   &     &        & a_{n-1} & b_{n-1}
\end{bmatrix}.
\end{equation}

The Thomas algorithm is a specialized form of Gaussian elimination that exploits
this tridiagonal structure to solve the system efficiently in $\mathcal{O}(n)$
time.

---

## Forward Sweep

During the forward sweep, the subdiagonal entries are eliminated and the system
is transformed into an equivalent upper-triangular system. We define modified
coefficients $b'_i$ and $d'_i$ recursively.

Initialization:
\begin{equation}
b'_0 = b_0, \qquad d'_0 = d_0.
\end{equation}

For $i = 1, 2, \ldots, n-1$, define
\begin{equation}
m_i = \frac{a_i}{b'_{i-1}}, \qquad
b'_i = b_i - m_i c_{i-1}, \qquad
d'_i = d_i - m_i d'_{i-1}.
\end{equation}

After the forward sweep, the system has been reduced to upper-triangular form.

---

## Back Substitution

The solution is then obtained by back substitution. Starting from the last
equation,
\begin{equation}
x_{n-1} = \frac{d'_{n-1}}{b'_{n-1}},
\end{equation}

and for $i = n-2, \ldots, 0$,
\begin{equation}
x_i = \frac{d'_i - c_i x_{i+1}}{b'_i}.
\end{equation}

In the following cells, we implement the Thomas algorithm step by step using
Python.

In [39]:
def thomas_forward_sweep(a, b, c, d):
    """
    Forward sweep of the Thomas algorithm using book notation.

    a[i] : subdiagonal (a[0] unused)
    b[i] : diagonal
    c[i] : superdiagonal (c[n-1] unused)
    d[i] : RHS

    Returns:
        b_prime : modified diagonal
        d_prime : modified RHS
    """
    n = len(b)

    b_prime = [0.0] * n
    d_prime = [0.0] * n

    # ---- i = 0 initialization ----
    b_prime[0] = b[0]
    d_prime[0] = d[0]

    if b_prime[0] == 0:
        raise ZeroDivisionError("Zero pivot at i = 0")

    # ---- forward sweep ----
    for i in range(1, n):
        m_i = a[i] / b_prime[i - 1]
        b_prime[i] = b[i] - m_i * c[i - 1]
        d_prime[i] = d[i] - m_i * d_prime[i - 1]

        if b_prime[i] == 0:
            raise ZeroDivisionError(f"Zero pivot at i = {i}")

    return b_prime, d_prime

In [40]:
def thomas_back_substitution(b_prime, c, d_prime):
    n = len(b_prime)
    x = [0.0] * n

    x[n - 1] = d_prime[n - 1] / b_prime[n - 1]

    for i in range(n - 2, -1, -1):
        print(i)
        x[i] = (d_prime[i] - c[i] * x[i + 1]) / b_prime[i]

    return x

In [41]:
import numpy as np

In [42]:
# Step 1: Define a 5x5 tridiagonal system Ax = d

import numpy as np

# Problem size
n = 5

# Subdiagonal (a_0 is unused)
a = np.array([0.0, -1.0, -1.0, -1.0, -1.0])

# Diagonal
b = np.array([4.0, 4.0, 4.0, 4.0, 4.0])

# Superdiagonal (c_{n-1} is unused)
c = np.array([-1.0, -1.0, -1.0, -1.0, 0.0])

# Right-hand side
d = np.array([3.0, 2.0, 2.0, 2.0, 3.0])

# Assemble the full matrix A (for verification only)
A = np.zeros((n, n))
for i in range(n):
    print(i)
    A[i, i] = b[i]
    if i > 0:
        A[i, i - 1] = a[i]
    if i < n - 1:
        A[i, i + 1] = c[i]

# Display results
print("Matrix A:")
print(A)

print("\nRight-hand side d:")
print(d)

0
1
2
3
4
Matrix A:
[[ 4. -1.  0.  0.  0.]
 [-1.  4. -1.  0.  0.]
 [ 0. -1.  4. -1.  0.]
 [ 0.  0. -1.  4. -1.]
 [ 0.  0.  0. -1.  4.]]

Right-hand side d:
[3. 2. 2. 2. 3.]


In [43]:
print(A[1,1])

4.0


In [44]:
print(A[1,2])

-1.0


In [45]:
# Step 2: Call the forward sweep

b_prime, d_prime = thomas_forward_sweep(a, b, c, d)

print("Modified diagonal b':")
print(np.array(b_prime))

print("\nModified right-hand side d':")
print(np.array(d_prime))

Modified diagonal b':
[4.         3.75       3.73333333 3.73214286 3.73205742]

Modified right-hand side d':
[3.         2.75       2.73333333 2.73214286 3.73205742]


In [46]:
# Step 3: Call the back substitution

x = thomas_back_substitution(b_prime, c, d_prime)

print("Solution x:")
print(np.array(x))

3
2
1
0
Solution x:
[1. 1. 1. 1. 1.]


In [47]:
# Step 4: Verification against NumPy

x_np = np.linalg.solve(A, d)

print("NumPy solution:")
print(x_np)

print("\nDifference (Thomas - NumPy):")
print(np.array(x) - x_np)

NumPy solution:
[1. 1. 1. 1. 1.]

Difference (Thomas - NumPy):
[0. 0. 0. 0. 0.]


### Forward Sweep (Thomas Algorithm)

Initialization:
\begin{equation}
b'_0 = b_0 = 4, \qquad d'_0 = d_0 = 3
\end{equation}

| $$i$$ | $$a_i$$ | $$b_i$$ | $$c_{i-1}$$ | $$m_i = \dfrac{a_i}{b'_{i-1}}$$ | $$b'_i = b_i - m_i c_{i-1}$$ | $$d'_i = d_i - m_i d'_{i-1}$$ |
|:----:|:-------:|:-------:|:-----------:|:--------------------------------:|:-----------------------------:|:-----------------------------:|
| $$0$$ | — | $$4$$ | — | — | $$4$$ | $$3$$ |
| $$1$$ | $$-1$$ | $$4$$ | $$-1$$ | $$-\dfrac{1}{4}$$ | $$\dfrac{15}{4}$$ | $$\dfrac{11}{4}$$ |
| $$2$$ | $$-1$$ | $$4$$ | $$-1$$ | $$-\dfrac{4}{15}$$ | $$\dfrac{56}{15}$$ | $$\dfrac{41}{15}$$ |
| $$3$$ | $$-1$$ | $$4$$ | $$-1$$ | $$-\dfrac{15}{56}$$ | $$\dfrac{209}{56}$$ | $$\dfrac{209}{56}$$ |